In [1]:
# dependencies
import sys
import os
from datetime import datetime
import pandas as pd

notebook_dir = os.getcwd()
project_root = None
for parent in [os.path.join(notebook_dir, os.pardir) for _ in range(5)]:
    if 'src' in os.listdir(parent):
        project_root = parent
        break
if project_root and project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# user lib
from src.data_processing.data_handler import RVnMacro, IV
from src.utilities.utilities import test_make_dir


Added /Users/oliverslott/Documents/MasterThesis/mt_code/notebooks/.. to sys.path


In [10]:
query_obj = RVnMacro(start=datetime(2005, 1, 1)).initialize_tickers().query_data()

No tickers specified. Setting default to S&P500 tickers.

503 tickers available as of 2024-03-09.
[**********************100%**********************] 514 of 514 completed

[**********************100%**********************]  86 of 86 completed

yfinance: download(show_errors=False) argument is deprecated and will be removed in future version. Do this instead to suppress error messages: logging.getLogger('yfinance').setLevel(logging.CRITICAL)


[*********************100%%**********************]  514 of 514 completed



[**********************100%**********************] 503 of 503 completed

[**********************100%**********************]   7 of 7 completed

In [12]:
# create data folder for plots and IV query:
plot_data_path = '../data/raw/plot_data'
plot_data_dir = test_make_dir(plot_data_path)
for freq in ('M', 'W'):
    for method in ('CC', 'GK', 'YZ'):
        df = query_obj.transform(sample_freq=freq, vol_measure=method)
        name = f'all_data_{freq}_{method}.parquet'
        full_path = plot_data_dir+f'/{name}'
        df.to_parquet(full_path)



Calculating commonality factor for the stock universe.
[**********************100%**********************] 514 of 514 completed


Columns with disallowed NaNs after the initial sequence: [('Market Commonality', 'AMCR'), ('Sector Commonality', 'AMCR')]
Dropping tickers: AMCR


Calculating commonality factor for the stock universe.
[**********************100%**********************] 514 of 514 completed


Columns with disallowed NaNs after the initial sequence: [('Market Commonality', 'AMCR'), ('Sector Commonality', 'AMCR')]
Dropping tickers: AMCR


Calculating commonality factor for the stock universe.
[**********************100%**********************] 514 of 514 completed


Columns with disallowed NaNs after the initial sequence: [('Market Commonality', 'AMCR'), ('Sector Commonality', 'AMCR')]
Dropping tickers: AMCR


Calculating commonality factor for the stock universe.
[**********************100%**********************] 514 of 514 completed


Columns with disallowed NaNs after the in

In [2]:
dividends = pd.read_csv('../data/processed/daily_dividends.csv')
rates_M = pd.read_parquet('../data/raw/plot_data/all_data_M_CC.parquet').xs(
    'Macro', 
    axis=1, 
    level=0
)[['US1M', 'US3M', 'US6M']].reset_index(drop=False).rename(columns={'index':'Date'})
rates_W = pd.read_parquet('../data/raw/plot_data/all_data_W_CC.parquet').xs(
    'Macro', 
    axis=1, 
    level=0
)[['US1M', 'US3M', 'US6M']].reset_index(drop=False).rename(columns={'index':'Date'})

In [3]:
# loading paths from external drive
iv_obj_M = IV(rates_M, dividends, 'm').initialize_paths()
iv_obj_W = IV(rates_W, dividends, 'w').initialize_paths()

In [4]:
# loading data onto local disk from external drive
iv_obj_M = iv_obj_M.load_data()
iv_obj_W = iv_obj_W.load_data()